## Langchain

- Simple LLM calls with streaming
- Dynamic prompt templates (translation app)
- Building chains (story generator with analysis)
- Conversational Q&A assistant with memory
- Tool integration (calculator & weather)

In [77]:
import langchain

In [78]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [79]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

##### Example 1: Simple LLM call with streaming

In [80]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage


In [81]:
model = init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000218363C8A70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000218363C9010>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [82]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
llm=ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000002183549C7C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000218345D3020>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [83]:
# Create messages
messages = [
    SystemMessage("You are a helpful AI assistant"),
    HumanMessage("What are top 3 benifits of using Langchain?")
]

In [84]:
response = model.invoke(messages)
response

AIMessage(content="Langchain is an open-source AI platform that enables users to create, customize, and control large language models (LLMs). Based on my knowledge, here are the top 3 benefits of using Langchain:\n\n1. **Customization and Control**: Langchain allows users to create custom LLMs by combining multiple models, APIs, and other data sources. This enables the creation of highly specialized models that cater to specific use cases, industries, or domains. Users can also fine-tune existing models to suit their needs, giving them more control over the output quality and accuracy.\n\n2. **Multi-Model Architecture**: Langchain's architecture enables users to integrate multiple models, APIs, and data sources to create a single, cohesive AI system. This allows for the creation of more robust and accurate models by leveraging the strengths of each individual component. For example, users can combine a large language model with a specialized model for a specific domain, creating a more

In [85]:
print(response.content)

Langchain is an open-source AI platform that enables users to create, customize, and control large language models (LLMs). Based on my knowledge, here are the top 3 benefits of using Langchain:

1. **Customization and Control**: Langchain allows users to create custom LLMs by combining multiple models, APIs, and other data sources. This enables the creation of highly specialized models that cater to specific use cases, industries, or domains. Users can also fine-tune existing models to suit their needs, giving them more control over the output quality and accuracy.

2. **Multi-Model Architecture**: Langchain's architecture enables users to integrate multiple models, APIs, and data sources to create a single, cohesive AI system. This allows for the creation of more robust and accurate models by leveraging the strengths of each individual component. For example, users can combine a large language model with a specialized model for a specific domain, creating a more effective and efficien

In [86]:
response1 = model.invoke([HumanMessage("What is machine learning?")])
print(response1.content)

Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time without being explicitly programmed.

The core idea behind machine learning is to enable machines to learn from experience and adapt to new situations, much like humans do. This is achieved by training machine learning models on large datasets, which allows them to identify patterns, relationships, and trends in the data.

There are several key characteristics of machine learning:

1. **Learning from data**: Machine learning models learn from data, rather than being programmed with explicit rules or instructions.
2. **Improvement over time**: Machine learning models can improve their performance over time, as they learn from new data and adapt to changing situations.
3. **Autonomy**: Machine learning models can operate independently, making decisions and taking actions witho

In [87]:
# streaming example
for chunk in model.stream(messages):
    print(chunk.content, end = "", flush=True)
    

Langchain is an AI platform that enables developers to build custom AI models and integrate them with various tools and services. Based on my knowledge cutoff, here are the top 3 benefits of using Langchain:

1. **Flexible AI Model Development**: Langchain provides a flexible and modular architecture for building custom AI models. It allows developers to integrate different AI components, such as language models, databases, and APIs, to create tailored solutions for specific use cases. This flexibility enables developers to build models that are optimized for their specific needs and requirements.

2. **Efficient Integration with Other Tools and Services**: Langchain has a vast ecosystem of integrations with other tools and services, including popular platforms like GitHub, Slack, and Google Cloud. This makes it easy for developers to integrate their Langchain models with existing workflows and systems, reducing the complexity and overhead associated with integrating new technologies.


##### Dynamic Prompt Templates

In [88]:
from langchain_core.prompts import ChatPromptTemplate

## create translation app
translation_template = ChatPromptTemplate.from_messages ([
    ("system", "You are a professional translator. Translate the following text: {text} to {source_language} to {target_language}. Maintain the tone and style"),
    ("user", "{text}")
])

## using the template
prompt = translation_template.invoke({
    "source_language":"English",
    "target_language":"Spanish",
    "text":"langchain makes building AI application increadibly easy!"
})
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a professional translator. Translate the following text: langchain makes building AI application increadibly easy! to English to Spanish. Maintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='langchain makes building AI application increadibly easy!', additional_kwargs={}, response_metadata={})])

In [89]:
translated_response = model.invoke(prompt)
print(translated_response.content)

langchain hace que construir aplicaciones de inteligencia artificial sea increíblemente fácil.


##### Building your First Chain


In [90]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def create_story_chain():
    # Template for story generation
    story_prompt = ChatPromptTemplate.from_messages([
        ("system","Your are a creative storyteller. Write a short and engaging story based on a given theme"),
        ("user","Theme: {theme}\n Main character: {character} \n Setting: {setting}")
    ])

    # Template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages ([
        ("system", "You are a literary critic. Analyze the following story and provide insights."),
        ("user", "{story}")
    ])

    # Build the chain - Method 1: Sequential execution using | operator
    story_chain = (
        story_prompt 
        | model 
        | StrOutputParser()
    )

    # Create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}
        
    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )

    return analysis_chain


In [91]:
chain = create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Your are a creative storyteller. Write a short and engaging story based on a given theme'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character} \n Setting: {setting}'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000218363C8A70>, async_client=<groq.resources.chat.completions.AsyncComplet

In [92]:
chain = create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Your are a creative storyteller. Write a short and engaging story based on a given theme'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character} \n Setting: {setting}'), additional_kwargs={})])
| ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000218363C8A70>, async_client=<groq.resources.chat.completions.AsyncComplet

In [93]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("Story and Analysis:")
print(result)

Story and Analysis:
This story has a futuristic and immersive setting, drawing inspiration from classic sci-fi narratives. Here's an in-depth analysis of the story, highlighting its strengths, weaknesses, and potential themes.

**Strengths:**

1. **Immersive world-building:** The author has created a vivid and detailed world, transporting the reader to the futuristic city of Neo-Eclipse. The description of towering skyscrapers, neon lights, and advanced technology effectively immerses the reader in this futuristic environment.
2. **Engaging protagonist:** Zeta, the small, curious robot, is an intriguing protagonist. His advanced artificial intelligence and ability to learn and adapt at an exponential rate make him a compelling character to follow.
3. **Mysterious plot:** The story raises intriguing questions about the city's ancient history, the existence of Echo, and the secrets hidden within the city's infrastructure. This mystery will likely keep readers engaged and curious about th